### Padding
- 자연어 처리에서 각 문장(문서)의 길이는 다룰 수 있음
- 그러나 언어모델은 고정된 길이의 데이터를 효율적으로 처리함
- 따라서 모든 문장의 **길이를 동일하게 맞춰주는 작업**이 필요함 === 패딩

**패딩 이점**
1. 일관된 입력 형식
2. 병렬 연산 최적화
3. 유연한 데이터 처리

- 딥러닝 모델을 사용할 때 CPU는 연산의 한계가 있어서 GPU를 사용하게 됨.
GPU는 속도가 빠른데 병렬 처리(동시처리)를 하기 때문임.
- 입력의 크기가 동일해야 병렬 처리가 가능함. (= 패딩 처리 = 병렬연산 최적화)
 

In [48]:
# 전처리가 완료된 형태의 문장들(임의 설정)
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'],
                          ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'],
                          ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'],
                          ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
                          ['barber', 'went', 'huge', 'mountain']]

#### 직접 구현 해보기
- 내부적으로 어떻게 동작하는지 원리에 대해 알아보자.

In [49]:
import torch
from collections import Counter

# 패딩 토크나이저 클래스 구현
class TokenizerForPadding:
    def __init__(self, num_words=None, oov_token='<OOV>'):
        self.num_words = num_words # 최대 단어 수
        self.oov_token = oov_token # OOV 토큰
        self.word_index = {} # 단어-인덱스 매핑
        self.index_word = {} # 인덱스-단어 매핑
        self.word_counts = Counter() # 단어-빈도수 매핑
    
    # 단어 사전 구축 메서드 구현 
    def fit_on_texts(self, texts):
        # 단어 빈도수 계산 
        for sentence in texts:  
            # self.word_counts.update(sentence)
            self.word_counts.update(word for word in sentence if word) # 빈 문자열 제외 처리 
        
        # 단어를 빈도수 기준으로 vocab 구성(num_words 고려)
        # most_common(n) : 빈도수 기준 상위 n개 단어 반환
        vocab = [self.oov_token] + [word for word, _ in self.word_counts.most_common(self.num_words)]
        
        # 단어-인덱스, 인덱스-단어 매핑 생성
        self.word_index = {word: i + 1 for i, word in enumerate(vocab)}
        self.index_word = {i + 1: word for i, word in enumerate(vocab)} 

    # 문장을 정수 시퀀스로 변환하는 메서드 구현
    def texts_to_sequences(self, texts):
        return [[self.word_index.get(word, self.word_index[self.oov_token]) for word in sentence] for sentence in texts]
        

In [50]:
tokenizer = TokenizerForPadding(num_words=15) # 단어 사전 크기 15 
tokenizer.fit_on_texts(preprocessed_sentences)  # 단어 사전 구축
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)  # 단어-인덱스 매핑 출력
sequences # 정수 시퀀스로 변환된 결과 출력 => 일관되지 않은 길이의 시퀀스

[[2, 6],
 [2, 9, 6],
 [2, 4, 6],
 [10, 3],
 [3, 5, 4, 3],
 [4, 3],
 [2, 5, 7],
 [2, 5, 7],
 [2, 5, 3],
 [8, 8, 4, 3, 11, 2, 12],
 [2, 13, 4, 14]]

In [51]:
# 패딩 함수 구현
#   - sequences: 정수 시퀀스 리스트
#   - maxlen: 패딩 후 시퀀스의 최대 길이 ( 데이터의 길이를 일관적으로 맞추기 위해 설정 )
#   - padding: 'pre' 또는 'post' (앞 또는 뒤에 패딩 추가)
#   - truncating: 'pre' 또는 'post' (앞 또는 뒤에서 시퀀스 자르기)
#   - value: 패딩에 사용할 값 , 기본값은 0, 빈 공간을 0으로 채움
def pad_sequences(sequences, maxlen, padding='pre', truncating='pre', value=0):
    
    # maxlen이 None인 경우, 시퀀스들 중 가장 긴 길이를 maxlen으로 설정
    if maxlen is None:
        maxlen = max(len(seq) for seq in sequences)
        
    padded_sequences = [] # 패딩된 시퀀스를 저장할 리스트
    
    # 각 시퀀스에 대해 패딩 또는 자르기 수행
    for seq in sequences:
        if len(seq) > maxlen:
            # 시퀀스 자르기
            if truncating == 'pre':
                seq = seq[-maxlen:]  # 음수 인덱스를 사용한 슬라이싱 (= 앞부분을 자름)
                # 예: [1,2,3,4,5]에서 maxlen=3이면 → [3,4,5] (앞의 1,2가 잘림)
                
            else:  # truncating == 'post'
                seq = seq[:maxlen]   # 양수 인덱스를 사용한 슬라이싱 (= 뒷부분을 자름)
                # 예: [1,2,3,4,5]에서 maxlen=3이면 → [1,2,3] (뒤의 4,5가 잘림)
                
        elif len(seq) < maxlen:
            # 패딩 추가
            pad_length = maxlen - len(seq)
            if padding == 'pre':
                seq = [value] * pad_length + seq  # 앞에 패딩(0) 추가
                # 예: [1,2,3]에서 maxlen=5이면 → [0,0,1,2,3] (앞에 0이 2개 추가)
                
            else:  # padding == 'post'
                seq = seq + [value] * pad_length  # 뒤에 패딩(0) 추가
                # 예: [1,2,3]에서 maxlen=5이면 → [1,2,3,0,0] (뒤에 0이 2개 추가)
        padded_sequences.append(seq)
    
    return torch.tensor(padded_sequences) 
    # 패딩된 시퀀스를 텐서로 반환 => 딥러닝 모델 입력으로 사용하기 위함
    # torch.tensor() 함수는 리스트를 PyTorch 텐서로 변환하는 역할을 함
    # 텐서는 다차원 배열로, 딥러닝 프레임워크에서 주로 사용됨

In [52]:
print(sequences)  # 패딩 전 시퀀스 출력

padded = pad_sequences(sequences, maxlen=None, padding='pre', truncating='pre', value=0)
print(padded)  # 패딩된 시퀀스 출력

# maxlen=None 설정 시, 가장 긴 시퀀스 길이에 맞춰 패딩이 자동으로 적용
#  - truncating='pre'로 설정했으므로, 긴 시퀀스의 앞부분이 잘림
#  - padding='pre'로 설정했으므로, 짧은 시퀀스의 앞부분에 0이 추가됨
#  - value=0이므로, 패딩 값은 0으로 채워짐
#  - 결과적으로 모든 시퀀스가 동일한 길이(7)로 맞춰짐

[[2, 6], [2, 9, 6], [2, 4, 6], [10, 3], [3, 5, 4, 3], [4, 3], [2, 5, 7], [2, 5, 7], [2, 5, 3], [8, 8, 4, 3, 11, 2, 12], [2, 13, 4, 14]]
tensor([[ 0,  0,  0,  0,  0,  2,  6],
        [ 0,  0,  0,  0,  2,  9,  6],
        [ 0,  0,  0,  0,  2,  4,  6],
        [ 0,  0,  0,  0,  0, 10,  3],
        [ 0,  0,  0,  3,  5,  4,  3],
        [ 0,  0,  0,  0,  0,  4,  3],
        [ 0,  0,  0,  0,  2,  5,  7],
        [ 0,  0,  0,  0,  2,  5,  7],
        [ 0,  0,  0,  0,  2,  5,  3],
        [ 8,  8,  4,  3, 11,  2, 12],
        [ 0,  0,  0,  2, 13,  4, 14]])


### keras Tokenizer 이용

In [53]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(preprocessed_sentences)  # 전처리된 문장들 출력

# 토큰화 및 정수 인코딩 
tokenizer = Tokenizer(num_words=15, oov_token='<OOV>') # 단어 사전 크기 15 , OOV 토큰 설정
tokenizer.fit_on_texts(preprocessed_sentences)  # 단어 사전 구축
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)  # 문장들을 정수 시퀀스로 변환
sequences # 정수 시퀀스로 변환된 결과 출력 => 일관되지 않은 길이의 시퀀스

print(sequences)  # 패딩 전 시퀀스 출력

# 패딩 적용
padded = pad_sequences(sequences, maxlen=None, padding='pre', truncating='pre', value=0)
print(padded)  # 패딩된 시퀀스 출력

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
[[2, 6], [2, 9, 6], [2, 4, 6], [10, 3], [3, 5, 4, 3], [4, 3], [2, 5, 7], [2, 5, 7], [2, 5, 3], [8, 8, 4, 3, 11, 2, 12], [2, 13, 4, 14]]
[[ 0  0  0  0  0  2  6]
 [ 0  0  0  0  2  9  6]
 [ 0  0  0  0  2  4  6]
 [ 0  0  0  0  0 10  3]
 [ 0  0  0  3  5  4  3]
 [ 0  0  0  0  0  4  3]
 [ 0  0  0  0  2  5  7]
 [ 0  0  0  0  2  5  7]
 [ 0  0  0  0  2  5  3]
 [ 8  8  4  3 11  2 12]
 [ 0  0  0  2 13  4 14]]


### [실습] 어린왕자 데이터 샘플 패딩처리

1. 텍스트 전처리 (토큰화/불용어처리/정제/정규화)
2. 정수 인코딩 Tokenizer (tensorflow.keras)
3. 패딩 처리 pad_sequences (tensorflow.keras)

In [54]:
# 어린왕자 데이터 샘플 텍스트
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [55]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))

# 데이터 전처리 함수 임포트
from text_preprocessing import (
    preprocess_text_for_encoding, 
)

# 1. 텍스트 전처리 (토큰화/불용어처리/정제/정규화)
vocab, preprocessed_sentences = preprocess_text_for_encoding(raw_text)

print(preprocessed_sentences)  # 전처리된 문장들 출력

# 2. 정수 인코딩 Tokenizer (tensorflow.keras)
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>') # 단어 사전 크기 1000 , OOV 토큰 설정
tokenizer.fit_on_texts(preprocessed_sentences)  # 단어 사전 구축
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)  # 문장들을 정수 시퀀스로 변환
print(sequences)  # 정수 시퀀스로 변환된 결과 출력 => 일관되지 않은 길이의 시퀀스

# 3. 패딩 처리 pad_sequences (tensorflow.keras)
padded = pad_sequences(sequences, maxlen=None, padding='post', truncating='post', value=0)
print(padded)  # 패딩된 시퀀스 출력

[['little', 'prince', 'written', 'antoine', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth'], ['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes'], ['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince'], ['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'lives', 'alone', 'rose', 'loves', 'deeply'], ['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets'], ['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox'], ['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior'], ['earth', 'little', 'prince', 'meets', 'various', 'creatures', 'including', 'fox', 'teaches', 'relationships', 'importance', 'taming', 'means', 'building', 'ties', 'others'], ['fox', 'famous', 'line', 'become', 'responsible', 'forever', 'tamed', 'resonates', 'prince', 'feelings', 'rose